# Library imports, configurations

In [1]:
import os
import re
import s3fs
import pandas as pd
import requests
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
from unidecode import unidecode
from wordcloud import WordCloud
import nltk
nltk.download('punkt')

from stop_words import get_stop_words
import unicodedata

from nltk.tokenize import word_tokenize
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch_dsl import connections
from bs4 import BeautifulSoup as bs

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [4]:
BUCKET = "jplaton/diffusion"
fs.ls(BUCKET)

['jplaton/diffusion/.keep',
 'jplaton/diffusion/ted',
 'jplaton/diffusion/visio_mel']

In [5]:
BUCKET_CLEM = "clementg/diffusion"
fs.ls(BUCKET_CLEM)

['clementg/diffusion/.keep',
 'clementg/diffusion/Fiche de Poste DINUM - francetransfert-3595970061- pdf.csv',
 "clementg/diffusion/Référentiel de l'organisation administrative de l'Etat.csv"]

# Functions

In [6]:
# Suppression des accents

def remove_accent(s):

    s = unidecode(str(s))
    return(s)

# Suppression de la ponctuation

def remove_punctuation(s):
    # On retire la ponctuation
 
    s = re.sub(r'[^A-Za-z0-9]',' ',str(s))
    return(s)

# Réduction de la casse

def tolower(s):
    s = ' '.join(str(s).lower().split())
    return(s)


# Sur cette base, on crée une petite fonction qui retire les stop words
stopwords = get_stop_words('french')   
stopwords = [unicodedata.normalize('NFKD',m).encode('ASCII', 'ignore').decode() for m in stopwords]

def remove_stopwords(s):
    s = [w for w in word_tokenize(s) if w not in stopwords]
    s = ' '.join(s)
    return(s)

def remove_accent_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].map(remove_accent)
    return df_c

def remove_punctuation_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].map(remove_punctuation)
    return df_c

def tolower_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].map(tolower)
    return df_c

# Scraping de annuaire service public pour aller chercher les services à aprtir des ids
# def get_service_from_scraping(service_id):
#     try:
#         url=f'https://lannuaire.service-public.fr/gouvernement/{service_id}'
#         response = requests.get(url)
#         html = response.content
#         soup = bs(html, "lxml")
#         return soup.title.get_text().replace(' - Annuaire | Service-public.fr', '')
#     except:
#         return ''

# DATA Download

In [7]:
FILE_PATH_TED = "ted"
FILE_PATH_TED_S3 = BUCKET + "/" + FILE_PATH_TED

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20230409.csv", mode="rb") as file_in:
    offres_2023_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20221225.csv", mode="rb") as file_in:   
    offres_2022_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/Bulloterie_10_Saison_2.xlsx", mode="rb") as file_in:  
    bulloterie_df =  pd.read_excel(file_in, sheet_name=0,header=[0,1])


/tmp/ipykernel_3125/4280061390.py:5: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2023_df = pd.read_csv(file_in, sep=";")
/tmp/ipykernel_3125/4280061390.py:8: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2022_df = pd.read_csv(file_in, sep=";")


In [8]:
with fs.open(BUCKET_CLEM+"/Fiche de Poste DINUM - francetransfert-3595970061- pdf.csv", mode="rb") as file_in:
    fdp_dinum_df = pd.read_csv(file_in, sep=";",encoding='Windows-1252')
with fs.open(BUCKET_CLEM+"/Référentiel de l'organisation administrative de l'Etat.csv", mode="rb") as file_in:
    orga_csv_df = pd.read_csv(file_in, sep=";",encoding='utf-8')

In [9]:
orga_sigle = orga_csv_df[['id','sigle (construction avec le nom)']].rename(columns={'sigle (construction avec le nom)':'sigle_calcul'})

In [10]:
with fs.open(FILE_PATH_TED_S3+"/dila_refOrga_admin_Etat_fr_20230505.json", mode="rb") as file_in:  
    orga =  json.load(file_in)

### Traitement référentiel services

In [11]:
orga_df = pd.merge(pd.json_normalize(orga,record_path='service'),orga_sigle,on='id',how='left')

In [12]:
ref_service_id_name = orga_df.set_index('id').to_dict()['nom']

In [13]:
ref_service_id_sigle = orga_df.set_index('id').to_dict()['sigle_calcul']

In [14]:
orga_df['services_fils_id'] = orga_df['hierarchie'].map(lambda x: [x[i]['service'] for i in range(len(x))])

In [15]:
orga_df['services_fils_name'] = orga_df['services_fils_id'].map(lambda x: [ref_service_id_name[n] for n in x if n in ref_service_id_name.keys()])

In [16]:
ref_pere_fils = orga_df.set_index('id').to_dict()['services_fils_id']

In [17]:
orga_df['arbo_service_id'] = ''
for k,v in ref_pere_fils.items():
   orga_df.loc[(orga_df['id'].isin(v))|(orga_df['id']==k),'arbo_service_id'] = orga_df.loc[(orga_df['id'].isin(v))|(orga_df['id']==k),'arbo_service_id'].apply(lambda x: [k]+v)

In [18]:
orga_df['arbo_service_name'] = orga_df['arbo_service_id'].map(lambda x: [ref_service_id_name[n] for n in x if n in ref_service_id_name.keys()])

In [19]:
ref_service_id_arbo = orga_df.set_index('id').to_dict()['arbo_service_name']

In [20]:
orga_df['arbo_service_sigle'] = orga_df['arbo_service_id'].map(lambda x: [ref_service_id_sigle[n] for n in x if n in ref_service_id_sigle.keys()])

In [47]:
orga_df[['id','nom','ancien_nom','type_organisme','sigle','services_fils_name','arbo_service_name','arbo_service_sigle']].to_csv('orga_with_arbo.csv',sep=';',index=False)

In [21]:
services_clean = ( orga_df
    .pipe(remove_accent_from_df,['nom','ancien_nom','type_organisme','sigle','services_fils_name','arbo_service_name','arbo_service_sigle'])
    .pipe(remove_punctuation_from_df,['nom','ancien_nom','type_organisme','sigle','services_fils_name','arbo_service_name','arbo_service_sigle'])
    .pipe(tolower_df,['nom','ancien_nom','type_organisme','sigle','services_fils_name','arbo_service_name','arbo_service_sigle'])
)

In [22]:
services_clean = services_clean[['id','nom','ancien_nom','type_organisme','sigle','services_fils_name','arbo_service_name','arbo_service_sigle']] 

### Traitement offres

In [23]:
offres_df = pd.concat([offres_2023_df,offres_2022_df])

In [24]:
offres_df.columns = offres_df.columns.map(remove_accent).map(remove_punctuation).map(tolower).map(lambda s: s.replace(' ', '_'))

In [25]:
offres_df_clean = ( offres_df
    .pipe(remove_accent_from_df,['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste'])
    .pipe(remove_punctuation_from_df,['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste'])
    .pipe(tolower_df,['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste'])
)

In [26]:
offres_df_clean = offres_df_clean[['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste']]

# Insertion des données dans Elastic Search

In [27]:
# Paramétrage du client
es_client = connections.create_connection(hosts=['http://elasticsearch-master:9200/'])

In [32]:
# Fontion qui transforme un dataframe en document/index à stocker dans ElasticSearch
def doc_generator(df,index_name):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": index_name,
                "_source": document,
            }

In [36]:
# Indexation des services
helpers.bulk(es_client, doc_generator(services_clean,"services"))

/tmp/ipykernel_3125/1014443642.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es_client, doc_generator(services_clean,"services"))


(7636, [])

In [28]:
def get_services(es,query,nb_results):
    fullsearch = es.search(index = "services", # l'index dans lequel on cherche
                       q = query, # notre requête textuelle
                       size = nb_results)
    res = [fullsearch['hits']['hits'][x]['_source']['id'] for x in range(nb_results)]
    return res

In [29]:
def add_services(df, list_cols,es,nb_results):
    df['service_id'] = df[list_cols].apply(lambda row: get_services(es,' '.join(row.values.astype(str)),nb_results), axis=1)
    return df

In [42]:
list_cols = ['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste']
offres_df_temp = offres_df_clean[offres_df_clean['metier'].str.contains("data scientist",na=False)|offres_df_clean['intitule_du_poste'].str.contains("data scientist",na=False)]
offres_with_services =  offres_df_temp.head(50).pipe(add_services,list_cols,es_client,3)
offres_with_services['service_nom'] = offres_with_services['service_id'].map(lambda x: [ref_service_id_name[n] for n in x if n in ref_service_id_name.keys()])
offres_with_services['service_arbo'] = offres_with_services['service_id'].map(lambda x: [ref_service_id_arbo[n] for n in x if n in ref_service_id_arbo.keys()])

/tmp/ipykernel_3125/4037462833.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch = es.search(index = "services", # l'index dans lequel on cherche
/tmp/ipykernel_3125/1241132570.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['service_id'] = df[list_cols].apply(lambda row: get_services(es,' '.join(row.values.astype(str)),nb_results), axis=1)
/tmp/ipykernel_3125/1349295690.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [78]:
# Indexation des offres enrichies des services
helpers.bulk(es_client, doc_generator(offres_with_services,"offres_with_services_ds_50"))

/tmp/ipykernel_3125/1862555769.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es_client, doc_generator(offres_with_services,"offres_with_services_ds_50"))


(50, [])

In [73]:
temp = offres_with_services[['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste','service_id']]
temp['description_offre'] = temp[['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste']].apply(lambda row: ' '.join(row.values.astype(str)),axis=1)
temp = (temp.drop(['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste'],axis=1)
        .explode('service_id')
        .groupby('service_id')
        .agg({'description_offre': lambda x: list(x)})
        .reset_index()
        .rename(columns={'service_id':'id','description_offre':'liste_offres_description'})
       )

/tmp/ipykernel_3125/2647205700.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['description_offre'] = temp[['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste']].apply(lambda row: ' '.join(row.values.astype(str)),axis=1)


In [75]:
services_with_offres = pd.merge(orga_df[['id','nom','ancien_nom','type_organisme','sigle','services_fils_name','arbo_service_name','arbo_service_sigle']],
                               temp,on='id',how='left')

In [80]:
# Indexation des services enrichies des offres
helpers.bulk(es_client, doc_generator(services_with_offres.fillna(''),"services_with_offres_ds_50"))

/tmp/ipykernel_3125/344794700.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es_client, doc_generator(services_with_offres.fillna(''),"services_with_offres_ds_50"))


BulkIndexError: 56 document(s) failed to index.

### Autres mode de recherche

In [19]:
from elasticsearch_dsl import Search
s = Search(index='offres').query('match',metier='charge')
r=s.execute()
df_results = pd.DataFrame((d.to_dict() for d in s.scan()))
 # Recherche dans l'ensemble des champs le meilleur écho (le plus pertinent) #
fullsearch_easy = es_client.search(index = "services", # l'index dans lequel on cherche
                       q = "direction generale des finances publiques dgfip	data scientis ssi dpn dtnum bureau si mission service a l utilisateur data scientist expert h f", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités
 # Recherche dans l'ensemble des champs le meilleur écho (le plus pertinent) #
fullsearch_scientist = es_client.search(index = "services", # l'index dans lequel on cherche
                       q = "centre hospitalier universitaire de tours	data scientist datascientist centre de donnees cliniques", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités
 # Recherche dans l'ensemble des champs le meilleur écho (le plus pertinent) #
fullsearch_engineer= es_client.search(index = "services", # l'index dans lequel on cherche
                       q = "service hydrographique et oceanographique de la marine	data engineer	ingenieur en traitement de l information conception developpement et experimentation d ia h f", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités
fullsearch_designer = es_client.search(index = "services", # l'index dans lequel on cherche
                       q = "caisse des depots et consignations	architecte technique expert conception et architecture de donnees d entreprise data designer h f	", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités

### Traitement Fiche de poste DNUM

In [14]:
# Traitement des colonnes
fdp_dinum_df.columns = fdp_dinum_df.columns.map(remove_accent).map(remove_punctuation).map(tolower).map(lambda s: s.replace(' ', '_'))
fdp_clean = ( fdp_dinum_df
    .pipe(remove_accent_from_df,['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences'])
    .pipe(remove_punctuation_from_df,['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences'])
    .pipe(tolower_df,['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences'])
)
fdp_clean = fdp_clean[['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences']] #fdp_clean.head(2)
